In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

import mimikit as mmk
import ipywidgets as W
import dataclasses as dtc
from typing import Optional, Callable, Any, Tuple, List, Dict
from IPython.display import display
UI = mmk.ui

cfg = mmk.TrainARMConfig()
# print(cfg.serialize())
display(mmk.views.train_arm_view(mmk.TrainARMConfig()))

Accordion(children=(VBox(children=(HBox(children=(Label(value='Batch Size: ', layout=Layout(margin='0 0 0 auto…

In [ ]:
options_style = dict(display="flex", margin="4px",
                     justify_content="space-between",
#                      justify_items="stretch",
                     align_content='stretch',
                     flex_flow='row',
                     width="75%",
#                     flex="2 0 auto"
                    )

buttons_style = dict()

data_acc = mmk.ConfigView(
            cfg.data,
            UI.Param(name="sources",
                    widget=UI.FileWidget().widget),
            UI.Param(name="sr",
              widget=W.ToggleButtons(description="Sample Rate: ", options=[16000, 22050, 32000, 44100], index=1,
                                     style=buttons_style,
                                     layout=options_style),
             ),
            UI.Param(name="n_fft", 
                  widget=W.ToggleButtons(description="FFT Size: ", options=[512, 1024, 2048, 4096], index=2, layout=options_style)),
            UI.Param(name="hop_length", 
                 widget=W.ToggleButtons(description="Overlapp: ", options=[1, 2, 4, 8], index=2, layout=options_style),
                 compute=lambda conf, ov: conf.n_fft // ov,),
            UI.Param(name="coordinate",
                  widget=W.ToggleButtons(description="Coordinates: ", options=["magnitude", "polar"], index=0, layout=options_style),
                 compute=lambda conf, co: co[:3]),
            UI.Param(name="db_path",
                    widget=W.Text(value="train.h5", description="Db Path: ", layout=options_style))
        ).as_widget(lambda children, **kwargs: W.Accordion([W.VBox(children=children)], **kwargs), titles=("Data",), selected_index=0)
display(data_acc)

In [16]:
cfg = mmk.WaveNet.Config()
label_layout = W.Layout(min_width="max-content", margin="0 0 0 auto")
param_layout = W.Layout(width="100%", margin="8px 0 8px 0")
view = UI.ConfigView(
    cfg,
    UI.Param(name='kernel_sizes',
             widget=UI.Labeled(
                 W.Label("kernel size",
                         layout=label_layout),
                 W.IntText(value=cfg.kernel_sizes[0],
                           layout={"width": "100px", }),
                 W.HBox(layout=param_layout)
             ),
             compute=lambda conf, ev: (ev,)),
    UI.Param(name="blocks",
             widget=UI.Labeled(
                 W.Label(value="layers per block",
                         layout=label_layout),
                 W.Text(value=str(cfg.blocks)[1:-1], layout=W.Layout(width="75%", )),
                 W.HBox(layout=param_layout), ),
             compute=lambda c, v: tuple(map(int, (s for s in v.split(",") if s not in ("", " "))))
             ),
    UI.Param(name="dims_dilated",
             widget=UI.pw2_widget(
                 W.Label(value="$N$ units per layer: ",
                         layout=label_layout, ),
                 W.Text(value=str(cfg.dims_dilated[0]),
                        layout=W.Layout(width="25%"), disabled=False),
                 W.Button(icon="plus", layout=W.Layout(width="25%")),
                 W.Button(icon="minus", layout=W.Layout(width="25%")),
                 W.HBox(layout=param_layout)),
             compute=lambda c, v: (int(v),)
             ),
            UI.Param(name='groups',
                 widget=UI.pw2_widget(
                 W.Label(value="groups of units",
                         layout=label_layout, ),
                 W.Text(value=str(cfg.groups),
                        layout=W.Layout(width="25%"), disabled=False),
                 W.Button(icon="plus", layout=W.Layout(width="25%")),
                 W.Button(icon="minus", layout=W.Layout(width="25%")),
                 W.HBox(layout=param_layout))
                    ),
    UI.Param(name="residuals_dim",
             widget=UI.yesno_widget(
                 label=W.Label(value="use residuals", layout=label_layout),
                 container=W.HBox(layout=param_layout),
                 initial_value=cfg.residuals_dim is not None,
                 buttons_layout=W.Layout(width="27.5%")),
             compute=lambda conf, ev: conf.dims_dilated[0] if ev else None
             ),
    UI.Param(name="skips_dim",
             widget=UI.yesno_widget(
                 label=W.Label(value="use skips", layout=label_layout),
                 container=W.HBox(layout=param_layout),
                 initial_value=cfg.residuals_dim is not None,
                 buttons_layout=W.Layout(width="27.5%")),
             compute=lambda conf, ev: conf.dims_dilated[0] if ev else None
             ),
).as_widget(lambda children, **kwargs: W.Accordion([W.VBox(children=children)], **kwargs),
            titles=("WaveNet Config",), selected_index=0, layout=W.Layout(margin="0 auto 0 0", width="500px"))


display(view)

Accordion(children=(VBox(children=(HBox(children=(Label(value='kernel size', layout=Layout(margin='0 0 0 auto'…

In [17]:
cfg

WaveNet.Config(io_spec=None, kernel_sizes=(2,), blocks=(4,), dims_dilated=(128,), dims_1x1=(), residuals_dim=None, apply_residuals=False, skips_dim=None, groups='16', act_f='Tanh', act_g='Sigmoid', pad_side=1, stride=1, bias=True, use_fast_generate=True)

In [ ]:



train_view = mmk.ConfigView(
    cfg,
    # ---------------------------
    UI.Param(
        name="data",
        widget=),
    # ---------------------------
    UI.Param(
        name="network",
        widget=),
    # ---------------------------
    UI.Param(
        name="training",
        widget=)
    # Layout
    # Config Validation
)



display(*train_view.widgets)

In [ ]:
mmk.demos.freqnet.main(cfg)